In [19]:
import os
import pandas as pd
"""엑셀 파일 데이터 변환"""
# ✅ Excel 파일이 있는 폴더 경로
folder_path = r"C:\VISSIM_Workspace\network_test\network01_output"

siml_count = 10  # 시뮬레이션 횟수

# ✅ 폴더 내 모든 .xlsx 파일 가져오기
#xlsx_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".xlsx")])
xlsx_files = ["7_output.xlsx"]
print(xlsx_files)
# ✅ 파일 처리
for file_name in xlsx_files:
    file_path = os.path.join(folder_path, file_name)

    # 🔹 Excel 파일 읽기
    df = pd.read_excel(file_path)

    # 🔹 특정 컬럼의 모든 값을 10으로 변경
    if "SIMRUN" in df.columns:
        df["SIMRUN"] = 10  # 모든 값을 10으로 설정
    if "ACCELERATION(ALL)" in df.columns:
        df["ACCELERATION(ALL)"] = round(df["ACCELERATION(ALL)"],2)  # 모든 값을 10으로 설정
    if "DIST(ALL)" in df.columns:
        df["DIST(ALL)"] = round(df["LENGTH(ALL)"],2)  # 모든 값을 10으로 설정
    if "LENGTH(ALL)" in df.columns:
        df["LENGTH(ALL)"] = round(df["LENGTH(ALL)"],2)  # 모든 값을 10으로 설정

    if "QUEUEDELAY(ALL)" in df.columns:
        df["QUEUEDELAY(ALL)"] = round(df["QUEUEDELAY(ALL)"],2)  # 모든 값을 10으로 설정
    if "SPEEDAVGARITH(ALL)" in df.columns:
        df["SPEEDAVGARITH(ALL)"] = round(df["SPEEDAVGARITH(ALL)"],2)  # 모든 값을 10으로 설정
    if "SPEEDAVGHARM(ALL)" in df.columns:
        df["SPEEDAVGHARM(ALL)"] = round(df["SPEEDAVGHARM(ALL)"],2)  # 모든 값을 10으로 설정
    df["SIMRUN"] = siml_count
 # 순서 재정의
    df = df[["SIMRUN", "TIMEINT", "DATACOLLECTIONMEASUREMENT", "ACCELERATION(ALL)", "DIST(ALL)",
         "LENGTH(ALL)", "VEHS(ALL)", "PERS(ALL)", "QUEUEDELAY(ALL)", "SPEEDAVGARITH(ALL)", "SPEEDAVGHARM(ALL)", "OCCUPRATE(ALL)"]]
    # 🔹 변경된 데이터 저장 (덮어쓰기)
    df.to_excel(file_path, index=False)
    print(f"✔ 파일 저장 완료: {file_path}")
print("✅ 모든 Excel 파일 업데이트 완료!")


['7_output.xlsx']
✔ 파일 저장 완료: C:\VISSIM_Workspace\network_test\network01_output\7_output.xlsx
✅ 모든 Excel 파일 업데이트 완료!
